## Imports & helper functions

In [53]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from timeit import default_timer as timer
from tqdm.notebook import tqdm
from torchinfo import summary
from pathlib import Path
import torch.nn.utils.prune as prune
import torch.nn.functional as F
from ptflops import get_model_complexity_info
import psutil
from model import SmokersBinaryClassification
from utils import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [54]:
# Calculate accuracy
def accuracy_fn(y_pred, y_true):
    # print(f'prediction {y_pred} true: {y_true} ')
    correct = torch.eq(y_true, y_pred).sum().item()
    return (correct / len(y_pred)) * 100

In [55]:
def print_train_time(start: float, end: float, device: torch.device = None):
    """Prints difference between start and end time.
    """
    total_time = end - start
    print(f'Train time on {device}: {total_time:.3f} seconds')
    # return total_time

In [56]:
def training_loop(model, X_train, y_train, X_test, y_test, epochs, loss_fn, optimizer):
    X_train, y_train, X_test, y_test = X_train.to(device), y_train.to(device), X_test.to(device), y_test.to(device)

    start_time = timer()

    for epoch in tqdm(range(epochs), desc='Training...'):
        model.train()

        # 1. Forward passs
        y_hat = model(X_train)
        loss = loss_fn(y_hat, y_train)    

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Calculate prediction accuracy
        y_train_pred = torch.round(torch.sigmoid(y_hat))        #if not using sigmoid on model output
        # y_train_pred = torch.round(y_hat)                     #prediction probabilities -> prediction labels
        acc = accuracy_fn(y_pred=y_train_pred, y_true=y_train)

        ### Testing
        model.eval()
        with torch.inference_mode():
            y_hat = model(X_test)
            test_loss = loss_fn(y_hat, y_test)

            y_test_pred = torch.round(torch.sigmoid(y_hat))
            # y_test_pred = torch.round(y_hat)
            test_acc = accuracy_fn(y_pred=y_test_pred, y_true=y_test)

        if epoch % 50 == 0 and epoch != 0             :
            print(f'Epoch: {epoch:04d} | Loss: {loss:.4f}, Acc: {acc:.2f}% | Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%')

    end_time = timer()
    return end_time - start_time, acc, test_acc
  

1. Initialize model
2. Save parameter values
3. Train network
4. Prune
5. Restore values from step 2 to pruned network
6. Repeat steps 3 - 5

## Data load and prep

In [57]:
dataset = pd.read_csv('dataset/smoking_driking_dataset_Ver01.csv')

In [58]:
dataset = dataset[dataset['SMK_stat_type_cd'] != 2]

In [59]:
dataset['sex'] = dataset['sex'].replace('Male', 0.0)
dataset['sex'] = dataset['sex'].replace('Female', 1.0)

In [60]:
dataset['SMK_stat_type_cd'] = dataset['SMK_stat_type_cd'].apply(lambda x: 1.0 if x == 3.0 else 0.0) # Smokers: 1 non smokers: 0

In [61]:
# Remove some input features from dataset
dataset = dataset.drop(['waistline','sight_left','sight_right','hear_left','hear_right','urine_protein','DRK_YN'], axis=1)

In [62]:
print(f'Number of smokers in the dataset: {len(dataset[dataset["SMK_stat_type_cd"]==1])}')
print(f'Number of non-smokers in the dataset: {len(dataset[dataset["SMK_stat_type_cd"]==0])}')

Number of smokers in the dataset: 213954
Number of non-smokers in the dataset: 602441


In [63]:
print(f'Number of smokers in the dataset: {len(dataset[dataset["SMK_stat_type_cd"]==1])}')
print(f'Number of non-smokers in the dataset: {len(dataset[dataset["SMK_stat_type_cd"]==0])}')

Number of smokers in the dataset: 213954
Number of non-smokers in the dataset: 602441


In [64]:
# Construct pandas dataframes for X and y values
df_X = dataset.drop(['SMK_stat_type_cd'], axis=1)
df_y = pd.DataFrame(dataset['SMK_stat_type_cd'])

In [65]:
# Train Test Split, Set X, y
X_train, X_test, y_train, y_test = train_test_split(df_X.to_numpy(), df_y.to_numpy(), test_size=0.2, random_state=1)

# Convert X features to float tensors
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

# Conver y labels to float tensors
y_train = torch.FloatTensor(y_train)
y_test = torch.FloatTensor(y_test)

## Training

In [66]:
model = SmokersBinaryClassification(input_features=16).to(device)

In [67]:
original_params = save_parameters(model)

In [68]:
# loss_fn = torch.nn.BCELoss().to(device)
loss_fn = torch.nn.BCEWithLogitsLoss().to(device)
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
optimizer= torch.optim.Adam(model.parameters(), lr=0.01)

In [69]:
for i in range(10):
    print(summary(model))
    time, train_acc, test_acc = training_loop(model, X_train, y_train, X_test, y_test, 400, loss_fn, optimizer)
    print(f'Iteration {i} training accuracy: {train_acc} | testing accuracy: {test_acc}')
    prune.l1_unstructured(model.stack[0], name='weight', amount=0.3)
    prune.l1_unstructured(model.stack[2], name='weight', amount=0.3)   
    restore_original_parameters(original_params, model.state_dict())

Layer (type:depth-idx)                   Param #
SmokersBinaryClassification              --
├─Sequential: 1-1                        --
│    └─Linear: 2-1                       1,088
│    └─ReLU: 2-2                         --
│    └─Linear: 2-3                       4,160
│    └─ReLU: 2-4                         --
│    └─Linear: 2-5                       65
Total params: 5,313
Trainable params: 5,313
Non-trainable params: 0


Training...:   0%|          | 0/400 [00:00<?, ?it/s]

Epoch: 0050 | Loss: 0.5003, Acc: 76.42% | Test Loss: 0.4920, Test Acc: 76.87%
Epoch: 0100 | Loss: 0.4415, Acc: 78.36% | Test Loss: 0.4381, Test Acc: 78.55%
Epoch: 0150 | Loss: 0.4214, Acc: 79.45% | Test Loss: 0.4192, Test Acc: 79.51%
Epoch: 0200 | Loss: 0.4929, Acc: 74.80% | Test Loss: 0.3985, Test Acc: 80.42%
Epoch: 0250 | Loss: 0.3916, Acc: 80.56% | Test Loss: 0.4061, Test Acc: 79.93%
Epoch: 0300 | Loss: 0.3680, Acc: 81.70% | Test Loss: 0.3666, Test Acc: 81.75%
Epoch: 0350 | Loss: 0.3666, Acc: 81.76% | Test Loss: 0.3626, Test Acc: 81.96%
Iteration 0 training accuracy: 81.65578549599152 | testing accuracy: 81.53712357376025
Layer (type:depth-idx)                   Param #
SmokersBinaryClassification              --
├─Sequential: 1-1                        --
│    └─Linear: 2-1                       781
│    └─ReLU: 2-2                         --
│    └─Linear: 2-3                       2,931
│    └─ReLU: 2-4                         --
│    └─Linear: 2-5                       65
Total 

Training...:   0%|          | 0/400 [00:00<?, ?it/s]

Epoch: 0050 | Loss: 0.3774, Acc: 80.44% | Test Loss: 0.3741, Test Acc: 80.59%
Epoch: 0100 | Loss: 0.3680, Acc: 81.18% | Test Loss: 0.3661, Test Acc: 81.22%
Epoch: 0150 | Loss: 0.3657, Acc: 81.45% | Test Loss: 0.3638, Test Acc: 81.48%
Epoch: 0200 | Loss: 0.3638, Acc: 81.65% | Test Loss: 0.3619, Test Acc: 81.67%
Epoch: 0250 | Loss: 0.3623, Acc: 81.82% | Test Loss: 0.3604, Test Acc: 81.94%
Epoch: 0300 | Loss: 0.3612, Acc: 81.93% | Test Loss: 0.3593, Test Acc: 82.03%
Epoch: 0350 | Loss: 0.3604, Acc: 82.02% | Test Loss: 0.3586, Test Acc: 82.13%
Iteration 1 training accuracy: 82.08572443486302 | testing accuracy: 82.21387931087281
Layer (type:depth-idx)                   Param #
SmokersBinaryClassification              --
├─Sequential: 1-1                        --
│    └─Linear: 2-1                       566
│    └─ReLU: 2-2                         --
│    └─Linear: 2-3                       2,071
│    └─ReLU: 2-4                         --
│    └─Linear: 2-5                       65
Total 

...

Epoch: 0050 | Loss: 0.3755, Acc: 80.70% | Test Loss: 0.3724, Test Acc: 80.83%
Epoch: 0100 | Loss: 0.3753, Acc: 80.72% | Test Loss: 0.3722, Test Acc: 80.80%
Epoch: 0150 | Loss: 0.3753, Acc: 80.73% | Test Loss: 0.3722, Test Acc: 80.81%
Epoch: 0200 | Loss: 0.3752, Acc: 80.72% | Test Loss: 0.3721, Test Acc: 80.81%
Epoch: 0250 | Loss: 0.3752, Acc: 80.72% | Test Loss: 0.3721, Test Acc: 80.82%
Epoch: 0300 | Loss: 0.3751, Acc: 80.72% | Test Loss: 0.3720, Test Acc: 80.83%
Epoch: 0350 | Loss: 0.3751, Acc: 80.73% | Test Loss: 0.3720, Test Acc: 80.83%
Iteration 7 training accuracy: 80.72409801627889 | testing accuracy: 80.83097030236588
Layer (type:depth-idx)                   Param #
SmokersBinaryClassification              --
├─Sequential: 1-1                        --
│    └─Linear: 2-1                       123
│    └─ReLU: 2-2                         --
│    └─Linear: 2-3                       300
│    └─ReLU: 2-4                         --
│    └─Linear: 2-5                       65
Total pa

Training...:   0%|          | 0/400 [00:00<?, ?it/s]

Epoch: 0050 | Loss: 0.3750, Acc: 80.73% | Test Loss: 0.3719, Test Acc: 80.84%
Epoch: 0100 | Loss: 0.3750, Acc: 80.73% | Test Loss: 0.3719, Test Acc: 80.85%
Epoch: 0150 | Loss: 0.3749, Acc: 80.74% | Test Loss: 0.3719, Test Acc: 80.85%
Epoch: 0200 | Loss: 0.3749, Acc: 80.73% | Test Loss: 0.3719, Test Acc: 80.86%
Epoch: 0250 | Loss: 0.3749, Acc: 80.73% | Test Loss: 0.3718, Test Acc: 80.87%
Epoch: 0300 | Loss: 0.3749, Acc: 80.73% | Test Loss: 0.3718, Test Acc: 80.88%
Epoch: 0350 | Loss: 0.3748, Acc: 80.74% | Test Loss: 0.3718, Test Acc: 80.86%
Iteration 8 training accuracy: 80.73175362416477 | testing accuracy: 80.86220518254031
Layer (type:depth-idx)                   Param #
SmokersBinaryClassification              --
├─Sequential: 1-1                        --
│    └─Linear: 2-1                       105
│    └─ReLU: 2-2                         --
│    └─Linear: 2-3                       229
│    └─ReLU: 2-4                         --
│    └─Linear: 2-5                       65
Total pa

Training...:   0%|          | 0/400 [00:00<?, ?it/s]

Epoch: 0050 | Loss: 0.3748, Acc: 80.73% | Test Loss: 0.3717, Test Acc: 80.88%
Epoch: 0100 | Loss: 0.3748, Acc: 80.73% | Test Loss: 0.3718, Test Acc: 80.86%
Epoch: 0150 | Loss: 0.3748, Acc: 80.73% | Test Loss: 0.3717, Test Acc: 80.87%
Epoch: 0200 | Loss: 0.3748, Acc: 80.73% | Test Loss: 0.3717, Test Acc: 80.86%
Epoch: 0250 | Loss: 0.3748, Acc: 80.73% | Test Loss: 0.3717, Test Acc: 80.86%
Epoch: 0300 | Loss: 0.3747, Acc: 80.74% | Test Loss: 0.3717, Test Acc: 80.86%
Epoch: 0350 | Loss: 0.3747, Acc: 80.74% | Test Loss: 0.3717, Test Acc: 80.86%
Iteration 9 training accuracy: 80.74094035362785 | testing accuracy: 80.86710477158728
